In [1]:
# Import important libraries

import numpy as np
import sympy as sp
import scipy.integrate as integrate

In [2]:
a1 = 24.001
a2 = 24
a3 = 12
a = np.array([a1, a2, a3])

eps11 = 0.001
eps22 = 0.001
eps33 = 0.001
eps12 = 0
eps13 = 0
eps23 = 0
app_strain = np.array([[eps11,eps12,eps13],[eps12,eps22,eps23],[eps13,eps23,eps33]])

nu = 0.3
E = 210


der_epsi = 0.001

delta = np.identity(3)
lambda_ = 1

In [3]:
# Solving the cubis equation of lambda (page 86 of Mura's book, equation 11.37)
def get_lambda(x,a):
    global lambda_
    sym = sp.Symbol('lambda_')
    equation = (x[0]**2)/(a[0]**2 + sym) + (x[1]**2)/(a[1]**2 + sym) + (x[2]**2)/(a[2]**2 + sym) - 1
    sol =  sp.solve(equation, sym)
    #print("Solution of Lambda: ",sol)

    # Convert sol to complex number
    sol = [complex(i) for i in sol]
    #print(sol)

    # Select the real part of the solution
    sol = [i.real for i in sol]
    #print(sol)
    
    sol = max(sol)
    if sol > 0:
        lambda_ = sol
    else:
        lambda_ = 0
    return lambda_
    #print("lambda = ",lambda_)

In [4]:
# Defining elliptic integrals and solving them. (page 85 of Mura's book, equation 11.36) lambda=0.. verified for lambda = 0
def I_(x,a, lamb = lambda_):
    #lambda_ = get_lambda(x,a)
    def integrand(s):
        return 1/np.sqrt((a[0]**2 + s)*(a[1]**2 + s)*(a[2]**2 + s))
    inte = integrate.quad(integrand, lamb, np.inf)[0]
    return inte*np.pi*2*a[0]*a[1]*a[2]

def Ii_(x,a, lamb = lambda_):
    #lambda_ = get_lambda(x,a)
    arr = np.zeros(3)
    for i in range(3):
        def integrand(s):
            return (1/(np.sqrt((a[0]**2 + s)*(a[1]**2 + s)*(a[2]**2 + s))*(a[i]**2 + s)))
        inte = integrate.quad(integrand, lamb, np.inf)[0]
        arr[i] = inte*np.pi*2*a[0]*a[1]*a[2]
    return arr

def Iij_(x,a, lamb = lambda_):
    #lambda_ = get_lambda(x,a)
    print("lambda = ",lamb)
    arr = np.zeros((3,3))
    for i in range(3):
        for j in range(3):
            def integrand(s):
                return (1/(np.sqrt((a[0]**2 + s)*(a[1]**2 + s)*(a[2]**2 + s))*(a[i]**2 + s)*(a[j]**2 + s)))
            inte = integrate.quad(integrand, lamb, np.inf)[0]
            arr[i,j] = inte*np.pi*2*a[0]*a[1]*a[2]
    return arr

In [5]:
x = np.array([25,0,0])
a = np.array([a1,a2,a3])
lambda_ = get_lambda(x,a)
print("lambda = ",lambda_)

lambda =  48.951999


In [6]:
Iij = Iij_(x,a,lambda_)
print("Iij = ",Iij)

lambda =  48.951999
Iij =  [[0.00258147 0.00258161 0.00570546]
 [0.00258161 0.00258176 0.00570581]
 [0.00570546 0.00570581 0.01348143]]


In [7]:
Ii = Ii_(x,a,lambda_)
print("Ii = ",Ii)

Ii =  [2.51348419 2.51360784 4.97852268]


In [8]:
# Compute Sijkl (page 88 of Mura's book, equation 11.42)

def get_Sijkl(x,a):
    global I, Ii, Iij
    Sijkl = np.zeros((3,3,3,3))
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    t1 = delta[i,j]*delta[k,l]*(2*nu*Ii[i] - Ii[k] + (a[i]**2)*Iij[k,i])
                    t2 = (delta[i,k]*delta[j,l] + delta[i,l]*delta[j,k])*((a[i]**2)*Iij[i,j] - Ii[j] + (1-nu)*(Ii[k] + Ii[l]))
                    Sijkl[i,j,k,l] = (t1+t2)/(8*np.pi*(1-nu))
                    print("S",i+1,j+1,k+1,l+1," = ",Sijkl[i,j,k,l])
    return Sijkl

In [9]:
# Compute derivatives of elliptic integrals (page 86 of Mura's book, equation 11.39)

#Compute derivative of Ii wrt to j direction
def Ii_j_(x,a):
    arr = np.zeros((3,3))
    for j in range(3):
        xp = x + der_epsi*delta[j]
        xm = x - der_epsi*delta[j]
        lambda_p = get_lambda(xp,a)
        lambda_m = get_lambda(xm,a)
        arr[j] = (I_(xp,a,lambda_p) - I_(xm,a,lambda_m))/(2*der_epsi)
    arr = np.moveaxis(arr,0,-1)
    return arr

# Compute derivative of Iij wrt to k direction
def Iij_k_(x,a):
    arr = np.zeros((3,3,3))
    for k in range(3):
        xp = x + der_epsi*delta[k]
        xm = x - der_epsi*delta[k]
        lambda_p = get_lambda(xp,a)
        lambda_m = get_lambda(xm,a)
        arr[k] = (Iij_(xp,a,lambda_p) - Iij_(xm,a,lambda_m))/(2*der_epsi)
    arr = np.moveaxis(arr,0,-1)
    return arr

# Compute double partial derivative of Iij wrt to k and l direction
def Iij_kl_(x,a):
    arr = np.zeros((3,3,3,3))
    for k in range(3):
        for l in range(3):
            xp_yp = x + der_epsi*delta[k] + der_epsi*delta[l]
            xm_yp = x - der_epsi*delta[k] + der_epsi*delta[l]
            xp_ym = x + der_epsi*delta[k] - der_epsi*delta[l]
            xm_ym = x - der_epsi*delta[k] - der_epsi*delta[l]
            lambda_pp = get_lambda(xp_yp,a)
            lambda_mp = get_lambda(xm_yp,a)
            lambda_pm = get_lambda(xp_ym,a)
            lambda_mm = get_lambda(xm_ym,a)
            arr[k,l] = (Iij_(xp_yp,a,lambda_pp) - Iij_(xm_yp,a,lambda_mp) - Iij_(xp_ym,a,lambda_pm) + Iij_(xm_ym,a,lambda_mm))/(4*der_epsi**2)
    arr = np.moveaxis(arr,[0,1],[-2,-1])
    return arr

# Compute derivative of Ii wrt to j and k direction
def Ii_jk_(x,a):
    arr = np.zeros((3,3,3))
    for j in range(3):
        for k in range(3):
            xp_yp = x + der_epsi*delta[j] + der_epsi*delta[k]
            xm_ym = x - der_epsi*delta[j] - der_epsi*delta[k]
            xp_ym = x + der_epsi*delta[j] - der_epsi*delta[k]
            xm_yp = x - der_epsi*delta[j] + der_epsi*delta[k]
            lambda_pp = get_lambda(xp_yp,a)
            lambda_mp = get_lambda(xm_yp,a)
            lambda_pm = get_lambda(xp_ym,a)
            lambda_mm = get_lambda(xm_ym,a)
            arr[j,k] = (Ii_(xp_yp,a,lambda_pp) - Ii_(xm_ym,a,lambda_mm) - Ii_(xp_ym,a,lambda_pm) + Ii_(xm_yp,a,lambda_mp))/(4*der_epsi**2)
    arr = np.moveaxis(arr,[0,1],[-2,-1])
    return arr

In [10]:
# Compute Dijkl (page 87 of Mura's book, equation 11.41)
def get_Dijkl(x,a,S):
    global Ii_j, Iij_k, Iij_kl, Ii_jk
    Dijkl = np.zeros((3,3,3,3))
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    t1 = 8*np.pi*(1-nu)*S[i,j,k,l] + 2*nu*delta[k,l]*x[i]*Ii_j[i,j]
                    t2 = (1+nu)*(delta[i,l]*x[k]*Ii_j[k,j] + delta[j,l]*x[k]*Ii_j[k,i] + delta[i,k]*x[l]*Ii_j[l,j] + delta[j,k]*x[l]*Ii_j[l,i])
                    t3 = -delta[i,j]*x[k]*(Ii_j[k,l] + a[i]**2*Iij_k[k,l,l]) - (delta[i,k]*x[j] + delta[j,k]*x[i])*(Ii_j[j,l] - a[i]**2*Iij_k[i,j,l])
                    t4 = -(delta[i,l]*x[j] + delta[j,l]*x[i])*(Ii_j[j,k] - a[i]**2*Iij_k[i,j,k]) - x[i]*x[j]*(Ii_jk[j,l,k] - a[i]**2*Iij_kl[i,j,l,k])
                    Dijkl[i,j,k,l] = (t1+t2+t3+t4)/(8*np.pi*(1-nu))
                    print("D",i+1,j+1,k+1,l+1," = ",Dijkl[i,j,k,l])
    return Dijkl

In [11]:
# Calculate epsilon tensor (page 85 of Mura's book, equation 11.33)
def get_epsilon(D,app_strain):
    epsilon = np.zeros((3,3))
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    epsilon[i,j] = epsilon[i,j] + D[i,j,k,l]*app_strain[k,l]
    return epsilon

In [12]:
# Calculate stress tensor for 3D
def get_stress(epsilon,E,nu,app_strain):
    epsilon = epsilon - app_strain
    sig11 = (E/((1+nu)*(1-2*nu)))*((1-nu)*epsilon[0,0] + nu*epsilon[1,1] + nu*epsilon[2,2])
    sig22 = (E/((1+nu)*(1-2*nu)))*(nu*epsilon[0,0] + (1-nu)*epsilon[1,1] + nu*epsilon[2,2])
    sig33 = (E/((1+nu)*(1-2*nu)))*(nu*epsilon[0,0] + nu*epsilon[1,1] + (1-nu)*epsilon[2,2])
    sig12 = (E/(1+nu))*epsilon[0,1]
    sig13 = (E/(1+nu))*epsilon[0,2]
    sig23 = (E/(1+nu))*epsilon[1,2]
    return np.array([[sig11,sig12,sig13],[sig12,sig22,sig23],[sig13,sig23,sig33]])

In [14]:
x = np.array([1,0,0])
lambda_ = get_lambda(x,a)
I = I_(x,a)
Ii = Ii_(x,a)
Iij = Iij_(x,a)
S = get_Sijkl(x,a)
Ii_j = Ii_j_(x,a)
Iij_k = Iij_k_(x,a)
Iij_kl = Iij_kl_(x,a)
Ii_jk = Ii_jk_(x,a)
Dijkl = get_Dijkl(x,a,S)
epsilon = get_epsilon(Dijkl,app_strain)
stress = get_stress(epsilon,E,nu,app_strain)
print("stress = ",stress)

lambda =  1
S 1 1 1 1  =  0.3936189799961544
S 1 1 1 2  =  0.0
S 1 1 1 3  =  0.0
S 1 1 2 1  =  0.0
S 1 1 2 2  =  0.04147994589149395
S 1 1 2 3  =  0.0
S 1 1 3 1  =  0.0
S 1 1 3 2  =  0.0
S 1 1 3 3  =  0.0013235486508849225
S 1 2 1 1  =  0.0
S 1 2 1 2  =  0.17607239524925689
S 1 2 1 3  =  0.0
S 1 2 2 1  =  0.17607239524925689
S 1 2 2 2  =  0.0
S 1 2 2 3  =  0.0
S 1 2 3 1  =  0.0
S 1 2 3 2  =  0.0
S 1 2 3 3  =  0.0
S 1 3 1 1  =  0.0
S 1 3 1 2  =  0.0
S 1 3 1 3  =  0.2800242854694469
S 1 3 2 1  =  0.0
S 1 3 2 2  =  0.0
S 1 3 2 3  =  0.0
S 1 3 3 1  =  0.2800242854694469
S 1 3 3 2  =  0.0
S 1 3 3 3  =  0.0
S 2 1 1 1  =  0.0
S 2 1 1 2  =  0.17607238987334434
S 2 1 1 3  =  0.0
S 2 1 2 1  =  0.17607238987334434
S 2 1 2 2  =  0.0
S 2 1 2 3  =  0.0
S 2 1 3 1  =  0.0
S 2 1 3 2  =  0.0
S 2 1 3 3  =  0.0
S 2 2 1 1  =  0.041485376053062156
S 2 2 1 2  =  0.0
S 2 2 1 3  =  0.0
S 2 2 2 1  =  0.0
S 2 2 2 2  =  0.39363591187607805
S 2 2 2 3  =  0.0
S 2 2 3 1  =  0.0
S 2 2 3 2  =  0.0
S 2 2 3 3  =  0.0013